In [43]:
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_squared_error

filename = 'ml-latest-small/ratings_train.csv'
df = pd.read_csv(filename)

def get_avg_ratings(trainfile):
    ret_dict = {}
    df = pd.read_csv(trainfile) 

    for index, row in df.iterrows():

        movie = int(row['movieId'])
        rating = row['rating']
        usr = int(row['userId'])
        if usr in ret_dict:
            ret_dict[usr-1].append(rating)
        else:
            ret_dict[usr-1] = [rating]

    for r in ret_dict:
        ret_dict[r] = sum(ret_dict[r])/len(ret_dict[r])
    return ret_dict
def get_movie_avg_ratings(trainfile):
    ret_dict = {}
    df = pd.read_csv(trainfile) 

    for index, row in df.iterrows():

        movie = int(row['movieId'])
        rating = row['rating']
        usr = int(row['userId'])
        if movie in ret_dict:
            ret_dict[movie].append(rating)
        else:
            ret_dict[movie] = [rating]

    for r in ret_dict:
        ret_dict[r] = sum(ret_dict[r])/len(ret_dict[r])
        
    return ret_dict
        
def get_usr_rat_dict(trainfile):
    ret_dict = {}
    df = pd.read_csv(trainfile)   

    for index, row in df.iterrows():

        movie = int(row['movieId'])
        rating = row['rating']
        usr = int(row['userId'])

        ret_dict[(usr, movie)] = rating 

    return ret_dict

def get_watched(trainfile):
    ret_dict = {}
    df = pd.read_csv(trainfile)  

    for index, row in df.iterrows():

        movie = int(row['movieId'])
        rating = row['rating']
        usr = int(row['userId'])

        if usr in ret_dict:
            ret_dict[usr].append(movie)
        else:
            ret_dict[usr] = [movie]

    return ret_dict


In [66]:

mov_map = {}

i = 0

list_movies = df_ratings['movieId'].unique()
c = len(list_movies)
idx_map = {}
for mv in list_movies:
    mov_map[mv] = i 
    idx_map[i] = mv
    i+=1


In [67]:
m_avgs = get_movie_avg_ratings('ml-latest-small/ratings_train.csv')
u_avgs = get_avg_ratings('ml-latest-small/ratings_train.csv')

In [68]:
global_avgs = df.replace(0, np.NaN).mean()
g_avg = sum(global_avgs)/len(global_avgs)

In [113]:
df_ratings = pd.read_csv('ml-latest-small/ratings_train.csv')
df = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

X = np.array(df)

rows = X.shape[0]
cols = X.shape[1]

In [105]:
for i in range(rows):
    u_a = u_avgs[i]
    for j in range(cols):
      
        movie = idx_map[j]
        
        m_a = m_avgs[movie]
        m_bias = m_a - g_avg
        u_bias = u_a - g_avg

#             X[i][j] -= m_bias
#             X[i][j] -= u_bias

KeyboardInterrupt: 

In [99]:
X.shape

(610, 9364)

In [114]:
X

array([[0. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [115]:

X = np.transpose(X)

u, s, vh = np.linalg.svd(X)

s = np.sqrt(s)
s_u = np.zeros(X.shape)
for i, v in enumerate(s):
    s_u[i, i] = v

s_v = np.diag(s)

r =4

A = np.matmul(u, s_u)[:, :r]
# print(s_v.shape, vh.shape)
B = np.matmul(s_v, vh)[:r, :]
AB = np.matmul(A, B)
loss = np.mean((AB - X)**2)

# print('A:', A)
# print('B:', B)
print('A.B:', AB)
print('MSE:', loss)


A.B: [[ 2.33411844e+00  1.44519218e-01  3.74226538e-02 ...  4.16548266e+00
   7.74604883e-01  2.33656225e+00]
 [ 1.04577311e+00  4.92414025e-02  2.54643410e-02 ...  2.79765628e+00
   5.17108864e-01  1.94459121e+00]
 [ 6.72566981e-01 -1.04705684e-01  2.78551575e-02 ...  1.41377689e+00
   2.57484254e-01  7.87278468e-02]
 ...
 [-6.36779979e-03  2.52777669e-03 -2.27056604e-04 ...  1.74963565e-03
  -2.07408188e-03  3.11068403e-02]
 [-6.36779979e-03  2.52777669e-03 -2.27056604e-04 ...  1.74963565e-03
  -2.07408188e-03  3.11068403e-02]
 [-7.57429424e-03  7.40030547e-03 -6.39516529e-04 ...  5.54361324e-03
  -3.68165273e-03  7.11563170e-02]]
MSE: 0.15422052200517386


In [116]:
newX = np.transpose(AB)
# for i in range(rows):
#     u_a = u_avgs[i]
#     for j in range(cols):
      
#         movie = idx_map[j]
        
#         m_a = m_avgs[movie]
#         m_bias = m_a - g_avg
#         u_bias = u_a - g_avg
        
        
#         newX[i][j] += m_bias
        
#         if newX[i][j] < 0:
#             newX[i][j] = 0
#         if newX[i][j] > 5:
#             newX[i][j] = 5
# #         newX[i][j] += u_bias

In [117]:
newX

array([[ 2.33411844e+00,  1.04577311e+00,  6.72566981e-01, ...,
        -6.36779979e-03, -6.36779979e-03, -7.57429424e-03],
       [ 1.44519218e-01,  4.92414025e-02, -1.04705684e-01, ...,
         2.52777669e-03,  2.52777669e-03,  7.40030547e-03],
       [ 3.74226538e-02,  2.54643410e-02,  2.78551575e-02, ...,
        -2.27056604e-04, -2.27056604e-04, -6.39516529e-04],
       ...,
       [ 4.16548266e+00,  2.79765628e+00,  1.41377689e+00, ...,
         1.74963565e-03,  1.74963565e-03,  5.54361324e-03],
       [ 7.74604883e-01,  5.17108864e-01,  2.57484254e-01, ...,
        -2.07408188e-03, -2.07408188e-03, -3.68165273e-03],
       [ 2.33656225e+00,  1.94459121e+00,  7.87278468e-02, ...,
         3.11068403e-02,  3.11068403e-02,  7.11563170e-02]])

In [118]:

d = pd.DataFrame(newX)
test_df = pd.read_csv('ml-latest-small/ratings_test_truth.csv')
preds = []
for index, row in test_df.iterrows():
    movieId = int(row['movieId'])
    usr = int(row['userId'])
   
    if movieId in mov_map:
        movie_idx = mov_map[movieId]
    
        val = newX[usr-1][movie_idx]
        if val > 5:
            val = 5
        if val < 0:
            val=0
        preds.append(val)
    else:
        preds.append(4)

test_df['pred'] = preds
y_true = test_df['rating']
y_pred = test_df['pred']

print("MSE of: ",mean_squared_error(y_true, y_pred))

MSE of:  11.42155750585313


In [146]:
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')
R_df = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

R = R_df.to_numpy()
user_ratings_mean = np.mean(R, axis = 1) - 3.2

R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [147]:
user_ratings_mean

array([-3.09582476, -3.18822501, -3.19023036, -3.12102016, -3.18354587,
       -3.08718634, -3.14950638, -3.18272316, -3.18457425, -3.1527972 ,
       -3.17511312, -3.18555121, -3.18837927, -3.18323735, -3.15212875,
       -3.16246401, -3.15454545, -3.00733237, -3.01149733, -3.11063348,
       -3.05145002, -3.16853147, -3.15459687, -3.15871041, -3.18714521,
       -3.19300699, -3.15074044, -3.0229638 , -3.16549774, -3.18344303,
       -3.17984369, -3.16061292, -3.13922254, -3.16976553, -3.1903332 ,
       -3.18375154, -3.19105306, -3.17418758, -3.15886466, -3.16009872,
       -3.12739613, -3.03864665, -3.1466269 , -3.18344303, -3.04096051,
       -3.18272316, -3.15603661, -3.1863225 , -3.19079597, -3.11135335,
       -3.06060263, -3.14014809, -3.18971617, -3.18971617, -3.19269848,
       -3.18200329, -3.03391608, -3.15505965, -3.15207733, -3.19156726,
       -3.18375154, -3.04635952, -3.09880708, -2.9996195 , -3.18591115,
       -3.05736323, -3.18529412, -2.78098519, -3.17932949, -3.17

In [139]:
R_demeaned

array([[ 3.89582476, -0.10417524,  3.89582476, ..., -0.10417524,
        -0.10417524, -0.10417524],
       [-0.01177499, -0.01177499, -0.01177499, ..., -0.01177499,
        -0.01177499, -0.01177499],
       [-0.00976964, -0.00976964, -0.00976964, ..., -0.00976964,
        -0.00976964, -0.00976964],
       ...,
       [ 2.23215755,  1.73215755,  1.73215755, ..., -0.26784245,
        -0.26784245, -0.26784245],
       [ 2.98755656, -0.01244344, -0.01244344, ..., -0.01244344,
        -0.01244344, -0.01244344],
       [ 4.50611888, -0.49388112, -0.49388112, ..., -0.49388112,
        -0.49388112, -0.49388112]])

In [140]:
R_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [142]:
sigma = np.diag(sigma)

In [143]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
all_user_predicted_ratings * 

In [144]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [145]:
preds_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.501444,-0.139015,-0.082080,0.079300,-0.158770,-0.587275,2.368039,-0.014790,-0.093695,-0.182211,...,-0.038935,-0.032500,-0.045369,-0.045369,-0.038935,-0.045369,-0.038935,-0.038935,-0.038935,-0.045416
606,2.849138,1.368651,0.341869,0.000534,-0.272603,1.529573,-0.078889,-0.013913,0.075251,1.398731,...,0.006789,0.006108,0.007471,0.007471,0.006789,0.007471,0.006789,0.006789,0.006789,-0.030371
607,2.161364,2.670920,2.128460,0.036007,0.128314,3.684387,-0.028717,0.195936,0.073591,2.406330,...,-0.044003,-0.041123,-0.046882,-0.046882,-0.044003,-0.046882,-0.044003,-0.044003,-0.044003,0.071872
608,0.780206,0.533649,0.096454,0.029945,0.087756,0.209604,0.078704,0.061626,-0.064337,1.242835,...,0.004355,0.004268,0.004442,0.004442,0.004355,0.004442,0.004355,0.004355,0.004355,-0.001346
